In [ ]:
import os
import re
import pandas as pd
import numpy as np

***Reading train and validation datasets***

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
val_data = pd.read_csv('/content/drive/MyDrive/val.csv')

In [ ]:
df['is_bad'].value_counts()

0    746221
1    238266
Name: is_bad, dtype: int64

In [ ]:
df = df[['description', 'category', 'is_bad']]
df

,description,category,is_bad
0,Продаем диван-кровать. Удобный механизм - евро...,Для дома и дачи,0
1,Кожух рулевой колонки DAF XF 94 (60066004)/\n ...,Транспорт,0
2,! Буст аккаунтов с ммр выше 1000ммр не беру ! ...,Услуги,1
3,Продам телевизор . Диагональ 450.наличие входа...,Бытовая электроника,1
4,Открытки-конверты ручной работы/\nВыполнены в ...,Хобби и отдых,0
...,...,...,...
984482,+79780979761/\nДенис/\nширину обода и вылет на...,Транспорт,1
984483,Зарядка и под в комплекте.,Хобби и отдых,0
984484,Белый стул Ikea /\nСостояние требует ремонта: ...,Для дома и дачи,0
984485,Бронирование с октября по январь 1010 г./\n ...,Недвижимость,0


In [ ]:
df['number of words'] = df['description'].apply(lambda n: len(n.split()))


In [ ]:
df

,description,category,is_bad,number of words
0,Продаем диван-кровать. Удобный механизм - евро...,Для дома и дачи,0,37
1,Кожух рулевой колонки DAF XF 94 (60066004)/\n ...,Транспорт,0,117
2,! Буст аккаунтов с ммр выше 1000ммр не беру ! ...,Услуги,1,38
3,Продам телевизор . Диагональ 450.наличие входа...,Бытовая электроника,1,57
4,Открытки-конверты ручной работы/\nВыполнены в ...,Хобби и отдых,0,7
...,...,...,...,...
984482,+79780979761/\nДенис/\nширину обода и вылет на...,Транспорт,1,8
984483,Зарядка и под в комплекте.,Хобби и отдых,0,5
984484,Белый стул Ikea /\nСостояние требует ремонта: ...,Для дома и дачи,0,28
984485,Бронирование с октября по январь 1010 г./\n ...,Недвижимость,0,278


In [ ]:
df = df.drop(df[df['number of words'] >= 725].index)
df = df.reset_index(drop=True)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df['is_bad'].value_counts()

0    746156
1    238264
Name: is_bad, dtype: int64

***Findind max length in category column***

In [ ]:
df['number of words'].max()

724

In [ ]:
df['number of words'].idxmax()

381061

In [ ]:
val_data = val_data[['description', 'category', 'is_bad']]
val_data['number of words'] = val_data['description'].apply(lambda n: len(n.split()))


In [ ]:
val_data['number of words'].max()

721

In [ ]:
df['category'].unique()

array(['Для дома и дачи', 'Транспорт', 'Услуги', 'Бытовая электроника',
       'Хобби и отдых', 'Личные вещи', 'Недвижимость', 'Животные',
       'Для бизнеса', 'Работа'], dtype=object)

In [ ]:
print('Транспорт ',df.query("category == 'Транспорт'")['is_bad'].value_counts())
print('Для дома и дачи ',df.query("category == 'Для дома и дачи'")['is_bad'].value_counts())
print('Услуги ',df.query("category == 'Услуги'")['is_bad'].value_counts())
print('Бытовая электроника ',df.query("category == 'Бытовая электроника'")['is_bad'].value_counts())
print('Хобби и отдых ',df.query("category == 'Хобби и отдых'")['is_bad'].value_counts())
print('Личные вещи ',df.query("category == 'Личные вещи'")['is_bad'].value_counts())
print('Недвижимость ',df.query("category == 'Недвижимость'")['is_bad'].value_counts())
print('Животные ',df.query("category == 'Животные'")['is_bad'].value_counts())
print('Для бизнеса ',df.query("category == 'Для бизнеса'")['is_bad'].value_counts())
print('Работа ',df.query("category == 'Работа'")['is_bad'].value_counts())

Транспорт  0    154402
1     54813
Name: is_bad, dtype: int64
Для дома и дачи  0    102017
1     30086
Name: is_bad, dtype: int64
Услуги  1    28430
0    28251
Name: is_bad, dtype: int64
Бытовая электроника  0    131093
1     21010
Name: is_bad, dtype: int64
Хобби и отдых  0    53813
1    10474
Name: is_bad, dtype: int64
Личные вещи  0    172146
1     27509
Name: is_bad, dtype: int64
Недвижимость  0    64101
1    36694
Name: is_bad, dtype: int64
Животные  1    17159
0    14262
Name: is_bad, dtype: int64
Для бизнеса  0    10203
1     1749
Name: is_bad, dtype: int64
Работа  0    15868
1    10340
Name: is_bad, dtype: int64


***Manually balancing the data***

In [ ]:
mask_transport_0 = df.loc[(df["category"] == "Транспорт") & (df["is_bad"] == 0)]
mask_transport_0 = mask_transport_0.head(54813)
mask_transport_1 = df.loc[(df["category"] == "Транспорт") & (df["is_bad"] == 1)]

mask_services_0 = df.loc[(df["category"] == "Услуги") & (df["is_bad"] == 0)]
mask_services_1 = df.loc[(df["category"] == "Услуги") & (df["is_bad"] == 1)]

mask_for_home_and_garden_0 = df.loc[(df["category"] == "Для дома и дачи") & (df["is_bad"] == 0)]
mask_for_home_and_garden_0 = mask_for_home_and_garden_0.head(30086)
mask_for_home_and_garden_1 = df.loc[(df["category"] == "Для дома и дачи") & (df["is_bad"] == 1)]

mask_appliances_0 = df.loc[(df["category"] == "Бытовая электроника") & (df["is_bad"] == 0)]
mask_appliances_0 = mask_appliances_0.head(21010)
mask_appliances_1 = df.loc[(df["category"] == "Бытовая электроника") & (df["is_bad"] == 1)]

mask_hobby_and_rest_0 = df.loc[(df["category"] == "Хобби и отдых") & (df["is_bad"] == 0)]
mask_hobby_and_rest_0 = mask_hobby_and_rest_0.head(10474)
mask_hobby_and_rest_1 = df.loc[(df["category"] == "Хобби и отдых") & (df["is_bad"] == 1)]

mask_personals_0 = df.loc[(df["category"] == "Личные вещи") & (df["is_bad"] == 0)]
mask_personals_0 = mask_personals_0.head(27509)
mask_personals_1 = df.loc[(df["category"] == "Личные вещи") & (df["is_bad"] == 1)]

mask_real_estate_0 = df.loc[(df["category"] == "Недвижимость") & (df["is_bad"] == 0)]
mask_real_estate_0 = mask_real_estate_0.head(36694)
mask_real_estate_1 = df.loc[(df["category"] == "Недвижимость") & (df["is_bad"] == 1)]

mask_animals_0 = df.loc[(df["category"] == "Животные") & (df["is_bad"] == 0)]
mask_animals_1 = df.loc[(df["category"] == "Животные") & (df["is_bad"] == 1)]

mask_for_business_0 = df.loc[(df["category"] == "Для бизнеса") & (df["is_bad"] == 0)]
mask_for_business_0 = mask_for_business_0.head(1749)
mask_for_business_1 = df.loc[(df["category"] == "Для бизнеса") & (df["is_bad"] == 1)]

mask_work_0 = df.loc[(df["category"] == "Работа") & (df["is_bad"] == 0)]
mask_work_0 = mask_work_0.head(10340)
mask_work_1 = df.loc[(df["category"] == "Работа") & (df["is_bad"] == 1)]

In [ ]:
df = pd.concat([mask_transport_0,mask_transport_1,
                mask_services_0, mask_services_1,
                mask_for_home_and_garden_0,mask_for_home_and_garden_1,
                mask_appliances_0,mask_appliances_1,
                mask_hobby_and_rest_0, mask_hobby_and_rest_1,
                mask_personals_0, mask_personals_1,
                mask_real_estate_0, mask_real_estate_1,
                mask_animals_0, mask_animals_1,
                mask_for_business_0, mask_for_business_1,
                mask_work_0, mask_work_1])

In [ ]:
df = df.sample(frac = 1)

In [ ]:
df = df.reset_index(drop=True)
df

,description,category,is_bad,number of words
0,Срочно сдаем./\n4-комнатная квартира в г. Коро...,Недвижимость,1,67
1,Блок питания Power RX6500 6100W предназначен д...,Бытовая электроника,0,388
2,"Жесткий диск 41Гб 35К LFF SAS HOT PLUG, 4.5-in...",Бытовая электроника,0,99
3,Продаю МФУ принтер HP Laser Jet 2156 MFP б/у в...,Бытовая электроника,1,27
4,"Продам новую электрическую плиту Лысьва, с пли...",Для дома и дачи,1,16
...,...,...,...,...
473447,Новые веганские кроссовки в ретро стиле. Встав...,Личные вещи,0,35
473448,Дверь заднее правое./\n /\nДверь заднее право...,Транспорт,0,163
473449,"ЗА НЕНАДОБНОСТЬЮ, Продам ОПТОМ полиэтиленовые ...",Хобби и отдых,1,455
473450,"ЭМ-65 Вента - орган-бас-синтезатор, имеет дин...",Хобби и отдых,0,135


***Importing NLTK for pre-processing purpose***

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def pre_process(data):
    ln = LancasterStemmer()
    corpus = []
    for i in range(0,len(data)):
        review = re.sub('[^а-яА-Я]', ' ', data['description'][i])
        review = review.lower()
        review = review.split()

        review = [ln.stem(word) for word in review if not word in stopwords.words('russian')]
        review = ' '.join(review)
        corpus.append(review)
    return corpus


In [ ]:
%%time
corpus = pre_process(df)

CPU times: user 45min 55s, sys: 6min 19s, total: 52min 14s
Wall time: 52min 53s


In [ ]:
%%time
corpus_valid = pre_process(val_data)

CPU times: user 2min 7s, sys: 17.5 s, total: 2min 24s
Wall time: 2min 26s


In [ ]:
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dropout

In [ ]:
voc_size=5000
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep_valid = [one_hot(words, voc_size) for words in corpus_valid]

Embedding


In [ ]:
sent_length = 730
embedded_docs=pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
embedded_docs_valid=pad_sequences(onehot_rep_valid, padding='pre', maxlen=sent_length)

In [ ]:
(embedded_docs[0])

In [ ]:
emb_vector_feat = 100
model = Sequential()
model.add(Embedding(voc_size, emb_vector_feat, input_length=sent_length))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu",))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 730, 100)          500000    
                                                                 
 dropout (Dropout)           (None, 730, 100)          0         
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1

In [ ]:
X_final  = np.array(embedded_docs)
y_final = np.array(df['is_bad'])
X_final.shape

(473452, 730)

In [ ]:
X_final_valid  = np.array(embedded_docs_valid)
y_final_valid = np.array(val_data['is_bad'])
X_final_valid.shape

(16237, 730)

In [ ]:
from sklearn.model_selection import train_test_split
x_tr, x_test, y_tr, y_test = train_test_split(X_final, y_final, test_size=0.1, random_state=42)
x_test.shape

In [ ]:
model.fit(X_final, y_final, validation_data=(X_final_valid, y_final_valid), epochs=5, batch_size=1000)

Epoch 1/5


In [ ]:
model.save('tf_model.h5')

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('my_tf_model.h5')

In [ ]:
y_pred = model.predict(X_final_valid)


508/508 [==============================] - 146s 279ms/step


In [ ]:
y_pred


array([[0.9741956 ],
       [0.24328479],
       [0.21277556],
       ...,
       [0.0375578 ],
       [0.03723534],
       [0.31432468]], dtype=float32)

In [ ]:
y_pred_not_float = tensorflow.cast(y_pred > 0.5, tensorflow.int32)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_pred_not_float, y_final_valid))
print(accuracy_score(y_pred_not_float, y_final_valid))

              precision    recall  f1-score   support

           0       0.82      0.92      0.87     11024
           1       0.77      0.59      0.67      5213

    accuracy                           0.81     16237
   macro avg       0.80      0.75      0.77     16237
weighted avg       0.81      0.81      0.80     16237

0.8111720145347047
